In [ ]:
from mhr.utils.utils import *

benchmark_statistic_file= "/mnt/petrelfs/songmingyang/songmingyang/data/mm/annotation/vqav2/statistics/val14_statistics.jsonl"

threshold_dict = {'object': 304, 'token': 120, 'co_occurrence': 24, 'what_word': 4895}
reverse_index_file_dict = {
    'object': '/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/LLaVA-Instruct-150K/reverse_index/llava_v1_5_mix665k_dino_stat_reverse_index.jsonl',
    'token': '/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/LLaVA-Instruct-150K/reverse_index/llava_v1_5_mix665k_token_reverse_index.jsonl',
    'co_occurrence': '/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/LLaVA-Instruct-150K/reverse_index/llava_v1_5_mix665k_co_occurrence_reverse_index.jsonl',
    'what_word': '/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/LLaVA-Instruct-150K/reverse_index/llava_v1_5_mix665k_what_word_reverse_index.jsonl'
}

token_input_file,object_input_file,co_occurrence_input_file,what_word_input_file = reverse_index_file_dict['token'],reverse_index_file_dict['object'],reverse_index_file_dict['co_occurrence'],reverse_index_file_dict['what_word']


In [2]:
def lookup_to_dict(obj,dictionary):
    return dictionary.get(obj,-1)

def get_90_index(data,sum,ratio=0.9):
    sum_90 = sum*ratio
    sum_temp = 0
    for i in range(len(data)):
        sum_temp += data[i][1]
        if sum_temp >= sum_90:
            return i
        
def get_scores(data,dictionary,threshold):
        scores = []
        metadata = []
        for item in data:
            score = lookup_to_dict(item,dictionary)
            if score >= 0:
                scores.append(score)
                metadata.append({"object":item,"score":score})
        avg_score = sum(scores)/len(scores) if len(scores) > 0 else 0
        max_score = max(scores) if len(scores) > 0 else 0
        avg_pass = avg_score >= threshold
        least_pass = max_score >= threshold
        return scores,metadata,avg_pass,least_pass

In [3]:
statistic_data = process_jsonl(benchmark_statistic_file)

for item in statistic_data:
    item["statistics"] = item["statistic"]


token_data = process_jsonl(token_input_file)
object_data = process_jsonl(object_input_file)
co_occurrence_data = process_jsonl(co_occurrence_input_file)
what_word_data = process_jsonl(what_word_input_file)

token_data.sort(key=lambda x: len(x["ids"]), reverse=True)
object_data.sort(key=lambda x: len(x["ids"]), reverse=True)
co_occurrence_data.sort(key=lambda x: len(x["ids"]), reverse=True)
what_word_data.sort(key=lambda x: len(x["ids"]), reverse=True)

token_data = [(x["object"],len(x["ids"])) for x in token_data]
object_data = [(x["object"],len(x["ids"])) for x in object_data]
co_occurrence_data = [(x["object"],len(x["ids"])) for x in co_occurrence_data]
what_word_data = [(x["object"],len(x["ids"])) for x in what_word_data]

token_sum = sum([x[1] for x in token_data])
object_sum = sum([x[1] for x in object_data])
co_occurrence_sum = sum([x[1] for x in co_occurrence_data])
what_word_sum = sum([x[1] for x in what_word_data])

token_threshold,object_threshold,co_occurrence_threshold,what_word_threshold = [0.9]*4 
token_90_loc = get_90_index(token_data,token_sum,token_threshold)
object_90_loc = get_90_index(object_data,object_sum,object_threshold)
co_occurrence_90_loc = get_90_index(co_occurrence_data,co_occurrence_sum,co_occurrence_threshold)
what_word_90_loc = get_90_index(what_word_data,what_word_sum,what_word_threshold)

token_dict = {x[0]:idx for idx,x in enumerate(token_data)}
object_dict = {x[0]:idx for idx,x in enumerate(object_data)}
co_occurrence_dict = {x[0]:idx for idx,x in enumerate(co_occurrence_data)}
what_word_dict = {x[0]:idx for idx,x in enumerate(what_word_data)}
for item in statistic_data:
        tokens = item["statistics"]["token"]
        objects = item["statistics"]["object"]
        co_occurrences = item["statistics"]["co_occurrence"]
        what_words = item["statistics"]["what_word"]
        
        token_scores,token_metadata,token_avg_pass,token_least_pass = get_scores(tokens,token_dict,token_90_loc)
        object_scores,object_metadata,object_avg_pass,object_least_pass = get_scores(objects,object_dict,object_90_loc)
        co_occurrence_scores,co_occurrence_metadata,co_occurrence_avg_pass,co_occurrence_least_pass = get_scores(co_occurrences,co_occurrence_dict,co_occurrence_90_loc)
        what_word_scores,what_word_metadata,what_word_avg_pass,what_word_least_pass = get_scores(what_words,what_word_dict,what_word_90_loc)
        distribution_item = dict(token=dict(scores=token_scores,metadata=token_metadata,avg_pass=token_avg_pass,least_pass=token_least_pass),
                                object=dict(scores=object_scores,metadata=object_metadata,avg_pass=object_avg_pass,least_pass=object_least_pass),
                                co_occurrence=dict(scores=co_occurrence_scores,metadata=co_occurrence_metadata,avg_pass=co_occurrence_avg_pass,least_pass=co_occurrence_least_pass),
                                what_word=dict(scores=what_word_scores,metadata=what_word_metadata,avg_pass=what_word_avg_pass,least_pass=what_word_least_pass))
        item["distribution"] = distribution_item

token_scores,token_metadata,token_avg_pass,token_least_pass = get_scores(tokens,token_dict,token_90_loc)

head_tail_pass_key = "avg_pass"
statistic_id_key = "question_id"

token_pass_cnt = 0
object_pass_cnt = 0
co_occurrence_pass_cnt = 0
what_word_pass_cnt = 0
all_pass_cnt = 0
pass_key = head_tail_pass_key
pass_ids = []
not_pass_ids = []
for item in statistic_data:
    token_pass,object_pass,co_occurrence_pass,what_word_pass = 0,0,0,0
    if item["distribution"]["token"][pass_key]:
        token_pass = 1
        token_pass_cnt += 1
    if item["distribution"]["object"][pass_key]:
        object_pass = 1
        object_pass_cnt += 1
    if item["distribution"]["co_occurrence"][pass_key]:
        co_occurrence_pass_cnt += 1
        co_occurrence_pass = 1
    if item["distribution"]["what_word"][pass_key]:
        what_word_pass_cnt += 1
        what_word_pass = 1
    if token_pass + object_pass + co_occurrence_pass + what_word_pass >= 1:
        all_pass_cnt += 1
        pass_ids.append(item[statistic_id_key])
    else:
        not_pass_ids.append(item[statistic_id_key])
pass_ids = pass_ids
not_pass_ids = not_pass_ids
tail_dict = {i:1 for i in pass_ids}
head_dict = {i:1 for i in not_pass_ids}
all_pass_cnt = all_pass_cnt


In [6]:
len(tail_dict)

47624

In [ ]:
def build_statistic_data(self):
    assert isinstance(statistic_files,list)
    if len(statistic_files) == 1:
        if statistic_files[0].endswith(".jsonl"):
            statistic_data = process_jsonl(statistic_files[0])
        elif statistic_files[0].endswith(".json"):
            statistic_data = load_json_file(statistic_files[0])
        if statistic_data[0].get("statistic",None) is not None:
            for item in statistic_data:
                item["statistics"] = item["statistic"]
        elif statistic_data[0].get("statistics",None) is None:
            raise ValueError("The statistic data should contain 'statistics' or 'statistic' key")
    else:
        token_data = process_jsonl(statistic_files[0])
        object_data = process_jsonl(statistic_files[1])
        what_word_data = process_jsonl(statistic_files[2])
        
        object_dict = {item[statistic_id_key]:str2list(item[statistic_object_keys[1]]) for item in object_data}
        token_dict = {item[statistic_id_key]:str2list(item[statistic_object_keys[0]]) for item in token_data}
        what_word_dict = {item[statistic_id_key]:str2list(item[statistic_object_keys[2]]) for item in what_word_data}
        
        statistic_data = []
        ids = list(set(list(object_dict.keys()) + list(token_dict.keys()) + list(what_word_dict.keys())))
        for id in ids:
            objects = list(set(object_dict.get(id,[])))
            tokens = token_dict.get(id,[])
            what_words = what_word_dict.get(id,[])
            co_occurrences = []
            for i in range(len(objects)):
                for j in range(i+1,len(objects)):
                    two_words = get_two_words(objects[i],objects[j])
                    co_occurrences.append(two_words)
            target_dict=dict(id=id,statistics=dict(object=objects,token=tokens,what_word=what_words,co_occurrence=co_occurrences))
            statistic_data.append(target_dict)
    
    def build_distribution_data(self):
        token_input_file,object_input_file,co_occurrence_input_file,what_word_input_file = distribution_reverse_index_files
        
        
        token_data = process_jsonl(token_input_file)
        object_data = process_jsonl(object_input_file)
        co_occurrence_data = process_jsonl(co_occurrence_input_file)
        what_word_data = process_jsonl(what_word_input_file)
        
        token_data.sort(key=lambda x: len(x["ids"]), reverse=True)
        object_data.sort(key=lambda x: len(x["ids"]), reverse=True)
        co_occurrence_data.sort(key=lambda x: len(x["ids"]), reverse=True)
        what_word_data.sort(key=lambda x: len(x["ids"]), reverse=True)
        
        token_data = [(x["object"],len(x["ids"])) for x in token_data]
        object_data = [(x["object"],len(x["ids"])) for x in object_data]
        co_occurrence_data = [(x["object"],len(x["ids"])) for x in co_occurrence_data]
        what_word_data = [(x["object"],len(x["ids"])) for x in what_word_data]
        
        token_data = token_data
        object_data = object_data
        co_occurrence_data = co_occurrence_data
        what_word_data = what_word_data

    def get_ratio_index(self):
        
        def get_90_index(data,sum,ratio=0.9):
            sum_90 = sum*ratio
            sum_temp = 0
            for i in range(len(data)):
                sum_temp += data[i][1]
                if sum_temp >= sum_90:
                    return i
                
        token_data,object_data,co_occurrence_data,what_word_data = token_data,object_data,co_occurrence_data,what_word_data
        token_sum = sum([x[1] for x in token_data])
        object_sum = sum([x[1] for x in object_data])
        co_occurrence_sum = sum([x[1] for x in co_occurrence_data])
        what_word_sum = sum([x[1] for x in what_word_data])
        
        token_threshold,object_threshold,co_occurrence_threshold,what_word_threshold = distribution_thresholds     
        token_90_loc = get_90_index(token_data,token_sum,token_threshold)
        object_90_loc = get_90_index(object_data,object_sum,object_threshold)
        co_occurrence_90_loc = get_90_index(co_occurrence_data,co_occurrence_sum,co_occurrence_threshold)
        what_word_90_loc = get_90_index(what_word_data,what_word_sum,what_word_threshold)
        print(f"token_{token_threshold}_loc:{token_90_loc}, object_{object_threshold}_loc:{object_90_loc}, co_occurrence_{co_occurrence_threshold}_loc:{co_occurrence_90_loc}, what_word_what_word_threshold_loc:{what_word_90_loc}")
        print(f"token_total {len(token_data)} object_total {len(object_data)} co_occurrence_total {len(co_occurrence_data)} what_word_total {len(what_word_data)}")
        
        
        token_dict = {x[0]:idx for idx,x in enumerate(token_data)}
        object_dict = {x[0]:idx for idx,x in enumerate(object_data)}
        co_occurrence_dict = {x[0]:idx for idx,x in enumerate(co_occurrence_data)}
        what_word_dict = {x[0]:idx for idx,x in enumerate(what_word_data)}
        loc_of_90=[token_90_loc,object_90_loc,co_occurrence_90_loc,what_word_90_loc]
    
    
    def get_head_tail_data_ids(self):
        def lookup_to_dict(obj,dictionary):
            return dictionary.get(obj,-1)

        def get_scores(data,dictionary,threshold):
            scores = []
            metadata = []
            for item in data:
                score = lookup_to_dict(item,dictionary)
                if score >= 0:
                    scores.append(score)
                    metadata.append({"object":item,"score":score})
            avg_score = sum(scores)/len(scores) if len(scores) > 0 else 0
            max_score = max(scores) if len(scores) > 0 else 0
            avg_pass = avg_score >= threshold
            least_pass = max_score >= threshold
            return scores,metadata,avg_pass,least_pass
        
        statistic_data = statistic_data
        token_90_loc,object_90_loc,co_occurrence_90_loc,what_word_90_loc = loc_of_90
        
        for item in statistic_data:
            tokens = item["statistics"]["token"]
            objects = item["statistics"]["object"]
            co_occurrences = item["statistics"]["co_occurrence"]
            what_words = item["statistics"]["what_word"]
            
            token_scores,token_metadata,token_avg_pass,token_least_pass = get_scores(tokens,token_dict,token_90_loc)
            object_scores,object_metadata,object_avg_pass,object_least_pass = get_scores(objects,object_dict,object_90_loc)
            co_occurrence_scores,co_occurrence_metadata,co_occurrence_avg_pass,co_occurrence_least_pass = get_scores(co_occurrences,co_occurrence_dict,co_occurrence_90_loc)
            what_word_scores,what_word_metadata,what_word_avg_pass,what_word_least_pass = get_scores(what_words,what_word_dict,what_word_90_loc)
            distribution_item = dict(token=dict(scores=token_scores,metadata=token_metadata,avg_pass=token_avg_pass,least_pass=token_least_pass),
                                    object=dict(scores=object_scores,metadata=object_metadata,avg_pass=object_avg_pass,least_pass=object_least_pass),
                                    co_occurrence=dict(scores=co_occurrence_scores,metadata=co_occurrence_metadata,avg_pass=co_occurrence_avg_pass,least_pass=co_occurrence_least_pass),
                                    what_word=dict(scores=what_word_scores,metadata=what_word_metadata,avg_pass=what_word_avg_pass,least_pass=what_word_least_pass))
            item["distribution"] = distribution_item
        
        token_pass_cnt = 0
        object_pass_cnt = 0
        co_occurrence_pass_cnt = 0
        what_word_pass_cnt = 0
        all_pass_cnt = 0
        pass_key = head_tail_pass_key
        pass_ids = []
        not_pass_ids = []
        for item in statistic_data:
            token_pass,object_pass,co_occurrence_pass,what_word_pass = 0,0,0,0
            if item["distribution"]["token"][pass_key]:
                token_pass = 1
                token_pass_cnt += 1
            if item["distribution"]["object"][pass_key]:
                object_pass = 1
                object_pass_cnt += 1
            if item["distribution"]["co_occurrence"][pass_key]:
                co_occurrence_pass_cnt += 1
                co_occurrence_pass = 1
            if item["distribution"]["what_word"][pass_key]:
                what_word_pass_cnt += 1
                what_word_pass = 1
            if token_pass + object_pass + co_occurrence_pass + what_word_pass >= 1:
                all_pass_cnt += 1
                pass_ids.append(item[statistic_id_key])
            else:
                not_pass_ids.append(item[statistic_id_key])
        pass_ids = pass_ids
        not_pass_ids = not_pass_ids
        tail_dict = {i:1 for i in pass_ids}
        head_dict = {i:1 for i in not_pass_ids}
        all_pass_cnt = all_pass_cnt
        print(f"token_pass_cnt:{token_pass_cnt}, object_pass_cnt:{object_pass_cnt}, \
      co_occurrence_pass_cnt:{co_occurrence_pass_cnt}, what_word_pass_cnt:{what_word_pass_cnt}, all_pass_cnt:{all_pass_cnt}")
        